<a href="https://colab.research.google.com/github/rezu98/colab/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install vaderSentiment
import matplotlib.pyplot as plt
import numpy as np
import nltk
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
nltk.download('stopwords')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from openpyxl import Workbook, load_workbook
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

import datetime as dt
import sys
import nltk
nltk.download('punkt')

In [ ]:
!pip install textstat
import textstat
from textstat import flesch_kincaid_grade, gunning_fog

In [ ]:
!pip install openpyxl

In [ ]:
file_path = 'results2.xlsx'

In [ ]:
### Sentiment Intensity Analyzer
# Example text
sample_text = "The Monetary Policy Board of the Bank of Korea decided today to leave the Base Rate unchanged at 3.50% for the intermeeting period. Although inflation is projected to continue its underlying trend of a slowdown, uncertainties regarding the future path of inflation and growth have risen significantly due to a prolongation of restrictive monetary policy stances in major countries and heightened geopolitical risks. In addition, it is forecast that the pace of inflation slowdown will moderate more than previously expected, and it is necessary to monitor household debt growth. The Board, therefore, sees that it is appropriate to maintain its current restrictive policy stance. Regarding the need to raise the Base Rate further, the Board will make a judgement while assessing the changes in domestic and external policy conditions. The currently available information suggests that uncertainties regarding economic and inflationary trends have increased across the global economy, driven by a prolongation of restrictive monetary policy stances in major countries and by the Israel-Hamas conflict. Global economic growth is projected to continue slowing. Inflation in major countries still remains high, though falling gradually, and upside risks have increased due to the rise in global oil prices. In global financial markets, volatility of major price variables has increased with government bond yields rising significantly and with the U.S. dollar strengthening considerably. Looking ahead, the Board sees global economic growth and global financial markets as likely to be affected by the movements of global oil prices and the global inflation slowdown, by monetary policy changes in major countries and their effects, and by developments in the Israel-Hamas conflict. Domestic economic growth has continued to improve at a modest pace owing to the easing of sluggishness in exports, although the recovery in private consumption has been somewhat slow. Labor market conditions have been generally favorable, as both a low unemployment rate and a robust increase in the number of persons employed have continued. Going forward, domestic economic growth is expected to improve gradually with the easing of the sluggishness in exports. GDP growth for the year is expected to be generally consistent with the August forecast of 1.4%. However, uncertainties surrounding the economic outlook are judged to be elevated, affected by heightened geopolitical risks and by the prolongation of restrictive monetary policy stances in major countries. Consumer price inflation has risen from August to 3.7% in September, due to the increase in the price of energy and of agricultural products. However, both core inflation (excluding changes in food and energy prices from the CPI) and short-term inflation expectations among the general public have stayed at 3.3% in September, the same as in August. Looking ahead, it is forecast that consumer price inflation will fall to the lower-3% range at the end of this year and will continue to gradually moderate in 2024. However, upside risks to inflation have increased due to the effects of higher global oil prices and exchange rates, and due to the Israel-Hamas conflict. Accordingly, it is judged that the timing of consumer price inflation converging on the target level is more likely to be delayed than previously expected. Meanwhile, core inflation is also projected to maintain its underlying slowing trend, owing to the weakening of demand-side pressures. However, the pace of the slowdown is likely to be more modest than previously forecast due to the continuing spillover effects of accumulated cost pressure. In financial and foreign exchange markets, volatility has increased as the U.S. Federal Reserve has signaled a prolongation of a high policy rate and as geopolitical risks have expanded. Long-term Korean Treasury bond yields and the Korean won to U.S. dollar exchange rate have risen significantly and stock prices have fallen. Meanwhile, the risks to some non-bank financial sectors have eased. Housing prices have continued their upward trend, especially in Seoul and its surrounding areas. Household loans have continued to increase, mainly driven by housing-related loans. The Board will continue to conduct monetary policy in order to stabilize consumer price inflation at the target level over the medium-term horizon as it monitors economic growth, while paying attention to financial stability. While domestic economic growth is forecast to gradually improve, uncertainties surrounding the policy decision have also risen. The Board, therefore, will maintain a restrictive policy stance for a considerable time with an emphasis on ensuring price stability, while making a judgement regarding the need to raise the Base Rate further. In this process, the Board will thoroughly assess the inflation slowdown, financial stability risks, economic downside risks, monetary policy changes in major countries, household debt growth, and developments in geopolitical risks."
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
# Perform sentiment analysis
scores = analyzer.polarity_scores(sample_text)
print(scores)

In [ ]:
### Sentiment Analyze

### by loughran and mcdonald(2019)

## loading master dictionary

def load_masterdictionary(file_path, print_flag=False, f_log=None, get_other=False):
    start_local = dt.datetime.now()
    # Setup dictionaries
    _master_dictionary = {}

    # Loop through words and load dictionaries
    with open(file_path, encoding="utf-8") as f:
        _total_documents = 0
        _md_header = f.readline()  # Consume header line
        print()
        for line in f:
            cols = line.rstrip('\n').split(',')
            word = cols[0].upper()  # Convert word to uppercase
            negative_value = int(cols[7])
            positive_value = int(cols[8])
            uncertainty_value = int(cols[9])
            _master_dictionary[word] = {'negative': negative_value, 'positive': positive_value, 'uncertainty': uncertainty_value}

    if print_flag:
        print('\r', end='')  # clear line
        print(f'\nMaster Dictionary loaded from file:\n  {file_path}\n')
        print(f'  master_dictionary has {len(_master_dictionary):,} words.\n')

    if get_other:
        return _master_dictionary, _md_header, _total_documents
    else:
        return _master_dictionary

In [ ]:
## evaluate sentiment indices

def evaluate_sentiment_indices(text, master_dictionary):
    words = nltk.word_tokenize(text)  # Tokenize the text into words
    words_uppercase = [word.upper() for word in words]  # Convert all words to uppercase
    total_words_count, negative_score, positive_score, uncertain_words_count = 0, 0, 0, 0
    negative_words_count, positive_words_count = 0, 0

    for word in words_uppercase:
        if word in master_dictionary:
            total_words_count += 1  # Increment the total word count
            if master_dictionary[word]['uncertainty'] != 0:
                uncertain_words_count += 1
            if master_dictionary[word]['negative'] > 0:
                negative_words_count += 1
            if master_dictionary[word]['positive'] > 0:
                positive_words_count += 1

    negative_index = negative_words_count / total_words_count  # Calculate the negative index
    positive_index = positive_words_count / total_words_count  # Calculate the positive index
    uncertainty_index = uncertain_words_count / total_words_count  # Calculate the uncertainty index

    return negative_index, positive_index, uncertainty_index, total_words_count, uncertain_words_count, negative_words_count, positive_words_count

if __name__ == '__main__':
    start = dt.datetime.now()
#    print(f'\n\n{start.strftime("%c")}\nPROGRAM NAME: {sys.argv[0]}\n')
    md = r'c.csv'  # Update with your file path
    master_dictionary = load_masterdictionary(md, True)

In [ ]:
### Financial Stability Dictionary
### by Correa, Ricardo & Keshav Garud & Juan M. Londono & Nathan Mislang, 2021. "Sentiment in Central Banks' Financial Stability Reports," Review of Finance, vol. 25(1), pages 85-120.

# Load the workbook
file_path_fs = 'f.xlsx'
workbook_fs = load_workbook(file_path_fs)
worksheet_fs = workbook_fs['1 FS Dictionary']

# Extract the words and their respective positive, negative, and assign values
fs_words = []
fs_positives = []
fs_negatives = []
fs_assigns = []

for row in worksheet_fs.iter_rows(min_row=2, min_col=1, max_col=4):
    if row[0].value:
        fs_words.append(row[0].value)
        fs_positives.append(row[1].value)
        fs_negatives.append(row[2].value)
        fs_assigns.append(row[3].value)

# Define the function to count the words in the fs dictionary in the original text and calculate the sum of positive and negative values
def count_fs_words(text, fs_words, fs_positives, fs_negatives):
    text_words = nltk.word_tokenize(text)  # Tokenize the text into words
    fs_positive_words = [word for word in text_words if word in fs_words and fs_positives[fs_words.index(word)] == 1]
    fs_negative_words = [word for word in text_words if word in fs_words and fs_negatives[fs_words.index(word)] == 1]
    fs_positive_sum = sum([fs_positives[fs_words.index(word)] for word in text_words if word in fs_words and fs_positives[fs_words.index(word)] == 1])
    fs_negative_sum = sum([fs_negatives[fs_words.index(word)] for word in text_words if word in fs_words and fs_negatives[fs_words.index(word)] == 1])
    return fs_positive_sum, fs_negative_sum, fs_positive_words, fs_negative_words

In [ ]:
## Readiability

def evaluate_readability(text):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)
    num_sentences = len(sentences)
    num_words = len(words)
    num_syllables = 0
    for word in words:
        num_syllables += textstat.syllable_count(word)
    return textstat.flesch_kincaid_grade(text), textstat.gunning_fog(text)

In [ ]:
### Example FOMC statement to evaluate - manually calculating

date = "2023.3"

fomc_statement = """

Good afternoon, the Vice-President and I welcome you to our press conference.

Inflation continues to decline but is still expected to remain too high for too long. We are determined to ensure that inflation returns to our two per cent medium-term target in a timely manner. In order to reinforce progress towards our target, the Governing Council today decided to raise the three key ECB interest rates by 25 basis points.

The rate increase today reflects our assessment of the inflation outlook in light of the incoming economic and financial data, the dynamics of underlying inflation, and the strength of monetary policy transmission. The September ECB staff macroeconomic projections for the euro area see average inflation at 5.6 per cent in 2023, 3.2 per cent in 2024 and 2.1 per cent in 2025. This is an upward revision for 2023 and 2024 and a downward revision for 2025. The upward revision for 2023 and 2024 mainly reflects a higher path for energy prices. Underlying price pressures remain high, even though most indicators have started to ease. ECB staff have slightly revised down the projected path for inflation excluding energy and food, to an average of 5.1 per cent in 2023, 2.9 per cent in 2024 and 2.2 per cent in 2025. Our past interest rate increases continue to be transmitted forcefully. Financing conditions have tightened further and are increasingly dampening demand, which is an important factor in bringing inflation back to target. With the increasing impact of our tightening on domestic demand and the weakening international trade environment, ECB staff have lowered their economic growth projections significantly. They now expect the euro area economy to expand by 0.7 per cent in 2023, 1.0 per cent in 2024 and 1.5 per cent in 2025.

Based on our current assessment, we consider that the key ECB interest rates have reached levels that, maintained for a sufficiently long duration, will make a substantial contribution to the timely return of inflation to our target. Our future decisions will ensure that the key ECB interest rates will be set at sufficiently restrictive levels for as long as necessary. We will continue to follow a data-dependent approach to determining the appropriate level and duration of restriction. In particular, our interest rate decisions will be based on our assessment of the inflation outlook in light of the incoming economic and financial data, the dynamics of underlying inflation, and the strength of monetary policy transmission.

The decisions taken today are set out in a press release available on our website.

I will now outline in more detail how we see the economy and inflation developing and will then explain our assessment of financial and monetary conditions.

Economic activity
The economy is likely to remain subdued in the coming months. It broadly stagnated over the first half of the year, and recent indicators suggest it has also been weak in the third quarter. Lower demand for the euro area’s exports and the impact of tight financing conditions are dampening growth, including through lower residential and business investment. The services sector, which had so far been resilient, is now also weakening. Over time, economic momentum should pick up, as real incomes are expected to rise, supported by falling inflation, rising wages and a strong labour market, and this will underpin consumer spending.

The labour market has so far remained resilient despite the slowing economy. The unemployment rate stayed at its historical low of 6.4 per cent in July. While employment grew by 0.2 per cent in the second quarter, momentum is slowing. The services sector, which has been a major driver of employment growth since mid-2022, is now also creating fewer jobs.

As the energy crisis fades, governments should continue to roll back the related support measures. This is essential to avoid driving up medium-term inflationary pressures, which would otherwise call for an even stronger monetary policy response. Fiscal policies should be designed to make our economy more productive and to gradually bring down high public debt. Policies to enhance the euro area’s supply capacity – which would be supported by the full implementation of the Next Generation EU programme – can help reduce price pressures in the medium term, while supporting the green transition. The reform of the EU’s economic governance framework should be concluded before the end of this year and progress towards Capital Markets Union should be accelerated.

Inflation
Inflation declined to 5.3 per cent in July but remained at that level in August, according to Eurostat’s flash estimate. Its decline was interrupted because energy prices rose compared with July. Food price inflation has come down from its peak in March but was still almost 10 per cent in August. In the coming months, the sharp price increases recorded in the autumn of 2022 will drop out of the yearly rates, thus pulling inflation down.

Inflation excluding energy and food fell to 5.3 per cent in August, from 5.5 per cent in July. Goods inflation declined to 4.8 per cent in August, from 5.0 per cent in July and 5.5 per cent in June, owing to better supply conditions, previous drops in energy prices, easing price pressures in the earlier stages of the production chain and weaker demand. Services inflation edged down to 5.5 per cent but was still kept up by strong spending on holidays and travel and by the high growth of wages. The annual growth rate of compensation per employee remained constant at 5.5 per cent in the second quarter of the year. The contribution of labour costs to annual domestic inflation increased in the second quarter, in part owing to weaker productivity, while the contribution of profits fell for the first time since early 2022.

Most measures of underlying inflation are starting to fall as demand and supply have become more aligned and the contribution of past energy price increases is fading out. At the same time, domestic price pressures remain strong.

Most measures of longer-term inflation expectations currently stand at around 2 per cent. But some indicators have increased and need to be monitored closely.

Risk assessment
The risks to economic growth are tilted to the downside. Growth could be slower if the effects of monetary policy are more forceful than expected, or if the world economy weakens, for instance owing to a further slowdown in China. Conversely, growth could be higher than projected if the strong labour market, rising real incomes and receding uncertainty mean that people and businesses become more confident and spend more.

Upside risks to inflation include potential renewed upward pressures on the costs of energy and food. Adverse weather conditions, and the unfolding climate crisis more broadly, could push food prices up by more than expected. A lasting rise in inflation expectations above our target, or higher than anticipated increases in wages or profit margins, could also drive inflation higher, including over the medium term. By contrast, weaker demand – for example due to a stronger transmission of monetary policy or a worsening of the economic environment outside the euro area – would lead to lower price pressures, especially over the medium term.

Financial and monetary conditions
Our monetary policy tightening continues to be transmitted strongly to broader financing conditions. Funding has again become more expensive for banks, as savers are replacing overnight deposits with time deposits that pay more interest and the ECB’s targeted longer-term refinancing operations are being phased out. Average lending rates for business loans and mortgages continued to increase in July, to 4.9 per cent and 3.8 per cent respectively.

Credit dynamics have weakened further. Loans to firms grew at an annual rate of 2.2 per cent in July, down from 3.0 per cent in June. Loans to households also grew less strongly, by 1.3 per cent, after 1.7 per cent in June. In annualised terms based on the last three months of data, household loans declined by 0.8 per cent, which is the strongest contraction since the start of the euro. Amid weak lending and the reduction in the Eurosystem balance sheet, the annual growth rate of M3 fell from 0.6 per cent in June to an all-time low of -0.4 per cent in July. In annualised terms over the past three months, M3 contracted by 1.5 per cent.

Conclusion
Inflation continues to decline but is still expected to remain too high for too long. We are determined to ensure that inflation returns to our two per cent medium-term target in a timely manner. In order to reinforce progress towards our target, the Governing Council today decided to raise the three key ECB interest rates by 25 basis points. Based on our current assessment, we consider that the key ECB interest rates have reached levels that, maintained for a sufficiently long duration, will make a substantial contribution to the timely return of inflation to our target. Our future decisions will ensure that the key ECB interest rates will be set at sufficiently restrictive levels for as long as necessary. We will continue to follow a data-dependent approach to determining the appropriate level and duration of restriction.

In any case, we stand ready to adjust all of our instruments within our mandate to ensure that inflation returns to our medium-term target and to preserve the smooth functioning of monetary policy transmission.

We are now ready to take your questions.




 """


 ## run!!!

if __name__ == '__main__':
    start = dt.datetime.now()
    md = r'c.csv'  # Update with your file path
    master_dictionary = load_masterdictionary(md, True)

    # Remove unnecessary line breaks and spaces
    processed_text = re.sub(r'\n+', ' ', fomc_statement)  # Replace multiple line breaks with a single space
    processed_text = re.sub(r' +', ' ', processed_text)  # Replace multiple spaces with a single space

    negative_index, positive_index, uncertainty_index, total_words_count, uncertain_words_count, negative_words_count, positive_words_count = evaluate_sentiment_indices(processed_text, master_dictionary)
    flesch_kincaid_grade, gunning_fog_index = evaluate_readability(processed_text)
    fs_positive_sum, fs_negative_sum, fs_positive_words, fs_negative_words = count_fs_words(processed_text, fs_words, fs_positives, fs_negatives)

    print(f"\nNumber of uncertain words: {uncertain_words_count}")
    print(f"Number of negative words: {negative_words_count}")
    print(f"Number of positive words: {positive_words_count}")
    print(f"Number of total words: {total_words_count}")
    print(f"Negative index for the FOMC statement: {negative_index:.2f}")
    print(f"Positive index for the FOMC statement: {positive_index:.2f}")
    print(f"Uncertainty index for the FOMC statement: {uncertainty_index:.2f}")
    print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_grade}")
    print(f"Gunning Fog Index: {gunning_fog_index}")
    print(f"FS positive sum: {fs_positive_sum}")
    print(f"FS negative sum: {fs_negative_sum}")
    print(f"FS_positive_words: {fs_positive_words}")
    print(f"FS_negative_words: {fs_negative_words}")

    # Printing the actual words defined as positive, negative, and uncertainty
    words = nltk.word_tokenize(processed_text)
    words_uppercase = [word.upper() for word in words]  # Convert all words to uppercase
    words_with_sentiment = [(word, master_dictionary[word]) for word in words_uppercase if word in master_dictionary]
    positive_words = [word for word, sentiment in words_with_sentiment if sentiment['positive'] > 0]
    negative_words = [word for word, sentiment in words_with_sentiment if sentiment['negative'] > 0]
    uncertain_words = [word for word, sentiment in words_with_sentiment if sentiment['uncertainty'] > 0]
    print(f"Positive words: {positive_words}")
    print(f"Negative words: {negative_words}")
    print(f"Uncertainty words: {uncertain_words}")


# Check if the file already exists, and load it if it does
try:
    workbook = load_workbook(file_path)
    sheet = workbook.active
except FileNotFoundError:
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(['Date', 'FOMC Statement', 'Positive Words', 'Negative Words', 'Uncertainty Words', 'Total Words', 'Positive Words List', 'Negative Words List', 'Uncertainty Words List', 'Flesch-Kincaid Grade Level', 'Gunning Fog Index', 'FS Positive_Counts', 'FS Negative_Counts', 'FS Positive Words', 'FS Negative Words'])


# Append results to the Excel file
sheet.append([date, processed_text, positive_words_count, negative_words_count, uncertain_words_count, total_words_count, ', '.join(positive_words), ', '.join(negative_words), ', '.join(uncertain_words), flesch_kincaid_grade, gunning_fog_index, fs_positive_sum, fs_negative_sum, '.join(fs_positive_words), ', '.join(fs_negative_words)'])

# Save the workbook
workbook.save(file_path)


In [ ]:
### FOMC statement to evaluate - using existing library - cleaning needed. imprecise.

# dataset = dataset, dataset_minutes, dataset_speechs
# dataset = dataset_statement

# Check if the file already exists, and load it if it does
try:
    workbook = load_workbook(file_path)
    sheet = workbook.active
except FileNotFoundError:
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(['Date', 'FOMC Statement', 'Positive Words', 'Negative Words', 'Uncertainty Words', 'Total Words', 'Positive Words List', 'Negative Words List', 'Uncertainty Words List', 'Flesch-Kincaid Grade Level', 'Gunning Fog Index', 'FS Positive_Counts', 'FS Negative_Counts', 'FS Positive Words', 'FS Negative Words'])

# Loop through the dataset and process each row
for index, row in dataset.iterrows():
    # Retrieve the date and FOMC statement from the current row
    date = row['date']  # Assuming the column name for the date is 'date'
    fomc_statement = row['statement']  # Assuming the column name for the statement is 'statement'

    # Your existing code
    start = dt.datetime.now()
    md = r'c.csv'  # Update with your file path
    master_dictionary = load_masterdictionary(md, True)

    # Remove unnecessary line breaks and spaces
    processed_text = re.sub(r'\n+', ' ', fomc_statement)
    processed_text = re.sub(r' +', ' ', processed_text)

    # Perform sentiment analysis, readability calculation, and other necessary calculations here

    negative_index, positive_index, uncertainty_index, total_words_count, uncertain_words_count, negative_words_count, positive_words_count = evaluate_sentiment_indices(processed_text, master_dictionary)
    flesch_kincaid_grade, gunning_fog_index = evaluate_readability(processed_text)
    fs_positive_sum, fs_negative_sum, fs_positive_words, fs_negative_words = count_fs_words(processed_text, fs_words, fs_positives, fs_negatives)

    # Printing the actual words defined as positive, negative, and uncertainty
    words = nltk.word_tokenize(processed_text)
    words_uppercase = [word.upper() for word in words]  # Convert all words to uppercase
    words_with_sentiment = [(word, master_dictionary[word]) for word in words_uppercase if word in master_dictionary]
    positive_words = [word for word, sentiment in words_with_sentiment if sentiment['positive'] > 0]
    negative_words = [word for word, sentiment in words_with_sentiment if sentiment['negative'] > 0]
    uncertain_words = [word for word, sentiment in words_with_sentiment if sentiment['uncertainty'] > 0]

    # Append results to the Excel file
    sheet.append([date, processed_text, positive_words_count, negative_words_count, uncertain_words_count, total_words_count, ', '.join(positive_words), ', '.join(negative_words), ', '.join(uncertain_words), flesch_kincaid_grade, gunning_fog_index, fs_positive_sum, fs_negative_sum, ', '.join(fs_positive_words), ', '.join(fs_negative_words)])

# Save the workbook
workbook.save(file_path)

In [ ]:
### Elminate stop words(conjunction, article, and preposition)

# Function to remove stopwords from a piece of text
def remove_stopwords(text):
    words = text.split()
    clean_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(clean_words)

In [ ]:
# Load the data from the Excel file
dataset_cleaning = pd.read_excel(file_path, engine='openpyxl')

# Get the stopwords list
stop_words = set(stopwords.words('english'))

# Apply the remove_stopwords function to the 'FOMC Statement' column and save the result to 'FOMC Statement_no_stopwords'
dataset_cleaning['FOMC Statement_no_stopwords'] = dataset_cleaning['FOMC Statement'].apply(remove_stopwords)

# Calculate the number of words
dataset_cleaning['words_cleaning'] = dataset_cleaning['FOMC Statement_no_stopwords'].str.split().apply(len)

# Calculate the number of occurrences of the word 'will'
dataset_cleaning['forward_words_will'] = dataset_cleaning['FOMC Statement'].str.count(r'\bwill\b')

# Calculate the number of occurrences of the word 'expect'
dataset_cleaning['forward_words_expect'] = dataset_cleaning['FOMC Statement'].str.count(r'\bexpect\b')

# Calculate the number of occurrences of the word 'may'
dataset_cleaning['forward_words_may'] = dataset_cleaning['FOMC Statement'].str.count(r'\bmay\b')

# Calculate the number of occurrences of the word 'might'
dataset_cleaning['forward_words_might'] = dataset_cleaning['FOMC Statement'].str.count(r'\bmight\b')

# Save the modified DataFrame to a new Excel file
with pd.ExcelWriter('new_results.xlsx', engine='openpyxl') as writer:
    dataset_cleaning.to_excel(writer, index=False, sheet_name='Sheet1')

In [ ]:
### Ziff's law and tf-idf score

# Load the excel
# file_path = 'results2.xlsx'
dataset_ziff = pd.read_excel(file_path)
# If Uncertainty Words List, it has vacant values. So use the following code.

# You may need to adjust the column names based on your dataset
dates = dataset_ziff['Date']
statements = dataset_ziff['Negative Words List']

# Create a dictionary to store words by their respective dates
words_by_date = {}

for index, statement in enumerate(statements):
    # Filter the statements to consider only those after 2019
    if dates[index].year < 2020:
        continue

    # Tokenize the text into words
    words = nltk.word_tokenize(statement)

    # Store the words with their respective date
    date = dates[index]
    if date not in words_by_date:
        words_by_date[date] = []
    words_by_date[date].extend(words)

# Now words_by_date dictionary contains the words stored by their respective dates after 2019

# Convert the words_by_date dictionary into a DataFrame
data = {'Date': list(words_by_date.keys()), 'Words': list(words_by_date.values())}
dataset_result_ziff = pd.DataFrame(data)

# Save the DataFrame to an Excel file
dataset_result_ziff.to_excel('result_word.xlsx', index=False)

In [ ]:
## Ziff's law

# Assuming you have already created the 'df' DataFrame
words_list = dataset_result_ziff['Words'].tolist()
words_flat = [word for sublist in words_list for word in sublist]

# Ziff's Law
word_counts = Counter(words_flat)
sorted_word_counts = sorted(word_counts.values(), reverse=True)
ranks = list(range(1, len(sorted_word_counts) + 1))
log_ranks = np.log(ranks)
log_counts = np.log(sorted_word_counts)

# Plot the scatter plot
plt.scatter(log_ranks, log_counts, color='b', alpha=0.5)
plt.title("Scatter plot for Ziff's Law")
plt.xlabel("Log of Ranks")
plt.ylabel("Log of Word Frequencies")
plt.show()

In [ ]:
## TF-idf score

# TF-IDF Scores
text = [' '.join(words) for words in words_list]

# Create the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)

# Get the feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Create a DataFrame for the TF-IDF scores
tfidf_df = pd.DataFrame(X.T.todense(), index=feature_names, columns=dataset_result_ziff['Date'])

# Save the results to Excel files
# ziffs_law_df = pd.DataFrame({'Ranks': ranks, 'Log Counts': log_counts, 'Log Ranks': log_ranks})
# ziffs_law_df.to_excel('ziffs_law_results.xlsx', index=False)
tfidf_df.to_excel('tfidf_scores.xlsx')

In [ ]:
### Relationship between words

# Load the excel
dataset_relationship = pd.read_excel(file_path)

# Define the word of interest
target_word = 'data-dependent'

# Initialize a counter for the occurrences of the target word
occurrences_counter = 0

# Iterate through the dataset to process the FOMC statements
for index, row in dataset_relationship.iterrows():
    date = row['Date']
    statement = row['FOMC Statement']

    # Check if the date falls within your specified time span
    if date.year < 2020:
        continue

    # Tokenize the FOMC statement into sentences
    sentences = nltk.sent_tokenize(statement)

    # Iterate through each sentence and check for occurrences of the target word
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        if target_word in words:
            index_of_word = words.index(target_word)

            # Extract the surrounding words
            words_around = words[max(0, index_of_word - 20): min(len(words), index_of_word + 25)]
            print(f"Date: {date}, {' '.join(words_around)}")


            # Increment the counter
            occurrences_counter += 1

# Print the total number of occurrences of the target word
print(f"Total occurrences of '{target_word}': {occurrences_counter}")

### LDA

In [ ]:
### LDA

from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

In [ ]:
# Load the dataset
dataset = pd.read_excel(file_path)

# Concatenate all text values in 'FOMC Statement' into a list of strings
concat_statements = dataset['FOMC Statement'].tolist()

# Tokenize the text data
tokenized_text = [word_tokenize(statement) for statement in concat_statements]

# Prepare the vocabulary
stop_words = set(stopwords.words('english'))

In [ ]:
# stop_word custom
custom_stop_words = stop_words.union(['chairman', 'Alan', 'Greenspan', 'Jr', 'Roger', 'Ferguson', 'Chairman', 'billion', 'Chair', 'pace', 'Reserve', 'time'])  # Add 'chairman' to the list of stop words

punctuation = set(string.punctuation)
texts = [
    [word for word in statement if word not in custom_stop_words and word not in punctuation]
    for statement in tokenized_text
]

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(texts)

# Filter out words that occur in less than 1 document or more than 80% of the documents
dictionary.filter_extremes(no_below=1, no_above=0.8)

# Bag-of-words representation of the documents
corpus = [dictionary.doc2bow(text) for text in texts]

# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=4, passes=10)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')

# Get the topics for each document
topics = [lda_model[corpus[i]] for i in range(len(corpus))]
print(topics)

In [ ]:
import matplotlib.pyplot as plt

# Convert the 'topics' list into a DataFrame
topics_df = pd.DataFrame(topics, columns=['Topic_0', 'Topic_1', 'Topic_2'])

# Combine 'dataset' with 'topics_df'
combined_df = pd.concat([dataset['Date'], topics_df], axis=1)

# Plot the time trends of the topics
plt.figure(figsize=(12, 6))
plt.plot(combined_df['Date'], combined_df['Topic_0'], label='Topic 0')
plt.plot(combined_df['Date'], combined_df['Topic_1'], label='Topic 1')
plt.plot(combined_df['Date'], combined_df['Topic_2'], label='Topic 2')
plt.plot(combined_df['Date'], combined_df['Topic_3'], label='Topic 3')

plt.xlabel('Date')
plt.ylabel('Topic Proportions')
plt.title('Topic Trends Over Time')
plt.legend()
plt.show()

In [ ]:
# Tokenize words and adjust the window size

vectorizer = CountVectorizer(ngram_range=(1, 10), stop_words='english')
X = vectorizer.fit_transform(concat_statements)
co_occurrence_matrix = X.T * X

# Convert the co-occurrence matrix to a DataFrame
co_occurrence_df = pd.DataFrame(co_occurrence_matrix.toarray(), index=vectorizer.get_feature_names_out(), columns=vectorizer.get_feature_names_out())

# Select the co-occurrence frequencies for the specified word
target_word = 'risks'
relevant_words = co_occurrence_df.loc[target_word]

# Print the co-occurrence frequencies of words related to the specified word
print(relevant_words)

In [ ]:
### Wordcloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Load the dataset
dataset = pd.read_excel(file_path)
# after 2019
# Concatenate all text values in 'FOMC Statement' into a single value
concat_statements = ' '.join(dataset['FOMC Statement'])

In [ ]:
# Generate a word cloud image(for 2020~)
wordcloud = WordCloud(width=700, height=400, background_color='white', colormap='viridis', stopwords=['and', 'the', 'to', 'of', 'in', 'for', 'a', 'Federal', 'Reserve', 'Committee', 'that', 'at', 'percent', 'williams', 'Vice', 'Voting', 'W Bowman', 'C Wiliams', 'Michelle', 'Jerome', 'H Powell', 'Bowman', 'W', 'H', 'Loretta', 'J', 'Last Update', 'Powell', 'Chair', 'Patrick', 'Harker', 'Randal', 'K', 'Lael', 'Brainard', 'has', 'on', 'as', 'have', 'month', 'per','its', 'Last', 'Update', 'Neel', 'Kashkari', 'target', 'range', 'funds', 'rate']).generate(concat_statements)

# Generate a word cloud image(before 2020~)
# wordcloud = WordCloud(width=700, height=400, background_color='white', colormap='viridis', stopwords=['and', 'the', 'to', 'of', 'in', 'for', 'a', 'Federal', 'Reserve', 'Committee', 'that', 'at', 'percent', 'williams', 'Vice', 'Voting', 'W Bowman', 'C Wiliams', 'Michelle', 'Jerome', 'H Powell', 'Bowman', 'W', 'H', 'Loretta', 'J', 'Last Update', 'Powell', 'Chair', 'Patrick', 'Harker', 'Randal', 'K', 'Lael', 'Brainard', 'has', 'on', 'as', 'have', 'month', 'per','its', 'Last', 'Update', 'Neel', 'Kashkari', 'funds', 'rate', 'Bernanke', 'S', 'Janet', 'L', 'Chairman', 'target', 'range', 'willaim']).generate(concat_statements)

# Display the generated word cloud image
plt.figure(figsize=(8, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

!!! Web crawling of FOMC Statements

In [ ]:
!pip install Fedtools

from FedTools import MonetaryPolicyCommittee
from FedTools import BeigeBooks
from FedTools import FederalReserveMins

In [ ]:
monetary_policy = MonetaryPolicyCommittee(
            main_url = 'https://www.federalreserve.gov',
            calendar_url = 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm',
            start_year = 2000,
            historical_split = 2014,
            verbose = True,
            thread_num = 10)

In [ ]:
dataset = monetary_policy.find_statements()
dataset.reset_index(inplace=True)
dataset.columns = ['date', 'statement']

In [ ]:
# from FedTools import BeigeBooks

# dataset = BeigeBooks().find_beige_books()
# BeigeBooks().pickle_data("directory.pkl")

In [ ]:
from FedTools import FederalReserveMins

dataset_minutes = FederalReserveMins().find_minutes()
dataset_minutes.reset_index(inplace=True)
dataset_minutes.columns = ['date', 'statement']

In [ ]:
import os
import sys
import urllib.request
client_id = "0MhoFcb7LR_ehhiyYObe" # 개발자센터에서 발급받은 Client ID 값
client_secret = "e0nXtPkgPV" # 개발자센터에서 발급받은 Client Secret 값
encText = urllib.parse.quote("반갑습니다")
data = "source=ko&target=en&text=" + encText
url = "https://openapi.naver.com/v1/papago/n2mt"
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request, data=data.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd

# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="AlanYR/mpr_tone_krfinbert")

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("AlanYR/mpr_tone_krfinbert")
model = AutoModelForSequenceClassification.from_pretrained("AlanYR/mpr_tone_krfinbert")

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/406M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/143k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/455k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# label 확인
print(model.config.label2id)
print(model.config.id2label)

{'negative': 0, 'neutral': 1, 'positive': 2}
{0: 'negative', 1: 'neutral', 2: 'positive'}


In [ ]:

# 3️⃣ 데이터 로드
df = pd.read_excel("seoul.xlsx")

# 날짜 컬럼 변환
df['date'] = pd.to_datetime(df['date'], format="%Y%m%d")

# 4️⃣ 제목 컬럼에서 "" 제거
df['subject'] = df['subject'].str.replace('"','').str.strip()

# 5️⃣ 예측
df['sentiment'] = df['subject'].apply(lambda x: pipe(x)[0]['label'])
df['score'] = df['subject'].apply(lambda x: pipe(x)[0]['score'])

# 6️⃣ 결과 확인
print(df[['date', 'subject', 'sentiment', 'score']].head())

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import torch
from tqdm.auto import tqdm

# 1️⃣ 모델과 pipeline 설정
model_name = "AlanYR/mpr_tone_krfinbert"

device = 0 if torch.cuda.is_available() else -1
print(f"✅ Using device: {device} ({'GPU' if device >= 0 else 'CPU'})")

# pipeline
pipe = pipeline("text-classification", model=model_name, device=device)

In [17]:
# 1️⃣ 파일명 변수
input_file = "NewsResult_20220701-20221231.xlsx"
output_file = input_file.replace(".xlsx", "_sentiment.xlsx")

# 2️⃣ 데이터 로드 (원래 엑셀 컬럼명 그대로 사용)
df = pd.read_excel(input_file, usecols=['일자', '제목'])

# 날짜 변환
df['일자'] = pd.to_datetime(df['일자'], format="%Y%m%d")

# 제목 전처리
df['제목'] = df['제목'].str.replace('"','').str.strip()

# 3️⃣ 예측 (progress bar 포함)
sentiments = []
scores = []

for text in tqdm(df['제목'], desc="Predicting"):
    output = pipe(text)[0]
    sentiments.append(output['label'])
    scores.append(output['score'])

df['sentiment'] = sentiments
df['score'] = scores

df_result = df[['일자', '제목', 'sentiment', 'score']]

# 4️⃣ 결과 확인
print(df_result.head())

# 5️⃣ 긍정비율 시계열 계산
## df['is_positive'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
## daily_positive = df.groupby(df['일자'].dt.to_period('D'))['is_positive'].mean()

# 6️⃣ 시계열 plot
## import matplotlib.pyplot as plt

## daily_positive.plot(figsize=(12,5), title="일자별 긍정비율 (%)")
## plt.ylabel("긍정비율")
## plt.show()


# 7️⃣ 저장
df_result.to_excel(output_file, index=False)
print(f"✅ 결과 저장 완료 → {output_file}")

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Predicting:   0%|          | 0/14642 [00:00<?, ?it/s]

          일자                                                 제목 sentiment  \
0 2022-12-31    20억 두고 ‘일진일퇴’ 잠실 엘리트 “19억대 매물 이제 안보여요” [부동산360]   neutral   
1 2022-12-31           ‘보이는 무엇이든 만들고 짓는다’- 민우식 건축가(下) [효효 아키텍트]   neutral   
2 2022-12-31  집주인도 “미치겠다”...올해 전세금 못돌려주는 비율 4% 될 듯 [나기자의 데이터...   neutral   
3 2022-12-31             유찰 또 유찰 “찬바람 경매 법정 삼수는 기본입니다” [부동산360]  positive   
4 2022-12-31            [추천!경매물건] 서울 양천구 신정동 ‘목동현대’ 9억7280만 원 등   neutral   

      score  
0  0.989956  
1  0.964588  
2  0.603526  
3  0.712873  
4  0.888944  
✅ 결과 저장 완료 → NewsResult_20220701-20221231_sentiment.xlsx
